In [9]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
import json
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import xmltodict
from tqdm import tqdm


In [10]:
nicotine = '/mnt/c/Users/cltng/OneDrive/Desktop/uniprot-compressed_true_download_true_format_json_query__28Nicotine_-2023.05.11-18.33.15.22.json'

In [11]:

with open(nicotine, 'r') as file:
    data = json.load(file)

# Now you can work with the JSON data stored in the 'data' variable.


In [12]:
ex = data['results'][0]

In [13]:
uniq = []
for i in data['results']:
  typ = (i['organism']['lineage'][11])

  if typ not in uniq:
    uniq.append(typ)
print(uniq)

['lamiids']


In [14]:
for i in ex['comments']:
    if i['commentType'] == 'SUBCELLULAR LOCATION':
        cell_location = (i['subcellularLocations'][0]['location']['value'])

In [15]:
ex = data['results'][0]
entry = {}

#protein names
entry['uniprot_id'] = ex['uniProtkbId']
entry['primary_accession'] = ex['primaryAccession']
entry['common_name'] = ex['organism']['commonName']
entry['species_name'] = ex['organism']['scientificName']
entry['sequence'] = ex['sequence']['value']

#subcellular location
for i in ex['comments']:
    if i['commentType'] == 'SUBCELLULAR LOCATION':
        entry['cell_location'] = (i['subcellularLocations'][0]['location']['value'])

#taxonomy
entry['superkingdom'] = ex['organism']['lineage'][0]
entry['kingdom'] = ex['organism']['lineage'][1]
entry['order'] = ex['organism']['lineage'][12]
entry['family'] = ex['organism']['lineage'][13]
entry['genus'] = ex['organism']['lineage'][14]

#protein properties
entry['properties'] = ex['features']

print(entry)

{'uniprot_id': 'DAO1_TOBAC', 'primary_accession': 'A0A1S3XSG2', 'common_name': 'Common tobacco', 'species_name': 'Nicotiana tabacum', 'sequence': 'MATTKQKVTAPSSSTAPCCPSTSILRREATAAVAGVGDGLQNWNNVPSVDDKQKKTASSALASLASTEPLSSNTSTKGIQIMTRAQTCHPLDPLSAAEISVAVATVRAAGETPEVRDGMRFIEVVLLEPDKSVVALADAYFFPPFQSSLMPRTKGGSLIPTKLPPRRARLIVYNKKTNETSIWIVELNEVHAAARGGHHRGKVISSNVVPDVQPPIDAQEYAECEAVVKSYPPFRDAMRRRGIDDLDLVMVDPWCVGYHSEADAPSRRLAKPLVFCRTESDCPMENGYARPVEGIYVLVDVQNMQIIEFEDRKLVPLPPADPLRNYTAGETRGGVDRSDVKPLHIIQPEGPSFRISGNYIEWQKWNFRIGFTPREGLVIHSVAYLDGSRGRRPIAHRLSFVEMVVPYGDPNDPHYRKNAFDAGEDGLGKNAHSLKRGCDCLGYIKYFDAHFTNFTGGVETTENCVCLHEEDHGMLWKHQDWRTGLAEVRRSRRLTVSFVCTVANYEYAFYWHFYQDGKIEAEVKLTGILSLGALQPGEYRKYGTTILPGLYAPVHQHFFVARMNMAVDCKPGEAHNQVVEVNVKVEEPGKENVHNNAFYAEETLLRSELQAMRDCDPFSARHWIVRNTRTVNRTGQLTGYKLVPGPNCLPLAGPEAKFLRRAAFLKHNLWVTQYAPGEEFPGGEFPNQNPRVGEGLASWVKQDRPLEESDIVLWYIFGITHVPRLEDWPVMPVEHIGFVLQPHGFFNCSPAVDVPPPSACDSESRDSDVTETSVAKSTATSLLAKL', 'cell_location': 'Peroxisome', 'superkingdom': 'Eukaryota', 'kin

In [16]:
for i in tqdm(ex['features']):
    print('type: ', i['type'])
    print('desc : ', i['description'])
    if 'ligand' in i:
        print('liga : ', i['ligand']['name'])

100%|██████████| 13/13 [00:00<00:00, 63255.16it/s]

type:  Chain
desc :  Diamine oxidase [copper-containing] 1, peroxisomal
type:  Active site
desc :  Proton acceptor
type:  Active site
desc :  Schiff-base intermediate with substrate; via topaquinone
type:  Binding site
desc :  
liga :  substrate
type:  Binding site
desc :  
liga :  substrate
type:  Binding site
desc :  
liga :  Cu cation
type:  Binding site
desc :  
liga :  Cu cation
type:  Binding site
desc :  
liga :  Mn(2+)
type:  Binding site
desc :  
liga :  Mn(2+)
type:  Binding site
desc :  
liga :  Cu cation
type:  Site
desc :  Microbody targeting signal
type:  Modified residue
desc :  2',4',5'-topaquinone
type:  Disulfide bond
desc :  


In [17]:
import os
from tqdm import tqdm
os.listdir('./blast_results')

[]

In [37]:
for ex in tqdm(data['results']):
    already_blasted = os.listdir('./blast_results/')
    if f"{ex['primaryAccession']}_blast_results.xml" not in already_blasted:
        protein_sequence_str = ex['sequence']['value']
        protein_sequence = SeqRecord(Seq(protein_sequence_str), id="my_protein", description="My protein sequence")
        result_handle = NCBIWWW.qblast("blastp", "nr", protein_sequence.seq)
        with open(f"./blast_results/{ex['primaryAccession']}_blast_results.xml", "w") as result_file:
            result_file.write(result_handle.read())

 39%|███▉      | 84/214 [17:11:14<10:16:08, 284.38s/it]   

In [18]:
protein_sequence_str = ex['sequence']['value']
protein_sequence = SeqRecord(Seq(protein_sequence_str), id="my_protein", description="My protein sequence")
result_handle = NCBIWWW.qblast("blastp", "nr", protein_sequence.seq)

In [31]:
NCBIXML.BlastParser(result_handle)

In [23]:
with open(f"./blast_results/{ex['primaryAccession']}_blast_results.json", "w") as result_file:
    result_file.write(result_handle)

TypeError: write() argument must be str, not _io.StringIO

In [32]:
def json_serializable(obj):
    if isinstance(obj, (list, tuple)):
        return [json_serializable(sub_obj) for sub_obj in obj]
    elif isinstance(obj, dict):
        return {key: json_serializable(value) for key, value in obj.items()}
    elif hasattr(obj, '__dict__'):
        return json_serializable(vars(obj))
    else:
        return obj

blast_records_json = json_serializable(list(blast_records))


In [33]:
blast_records_json

[]

ValueError: Your XML file was empty